If you do not have sentinelsat package installed on your machine run the command below

In [ ]:
import sys
!{sys.executable} -m pip install sentinelsat

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

import zipfile
import rasterio 
from rasterio.mask import mask
import os
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

In [2]:
os.chdir('..')

In [3]:
os.getcwd()

'/Users/ksunil/Desktop/Karan/Codebase'

In [28]:
user = 'karans04'
password = 'igpp2019'
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [5]:
reef_gjson_fp = os.path.join('data','Moce' ,'Moce.geojson')
reef_footprint = geojson_to_wkt(read_geojson(reef_gjson_fp))

In [17]:
products = api.query(reef_footprint,date = ('20190101', '20200331'),platformname = 'Sentinel-2',area_relation = 'Intersects',\
                     processinglevel = 'Level-2A',cloudcoverpercentage = (0,10),order_by = 'cloudcoverpercentage')

In [18]:
sentinel_path = os.path.join('data','Nasau', 'SAFE files')
if not os.path.exists(sentinel_path):
    os.makedirs(sentinel_path)

In [22]:
for k,v in products.items():
    api.download(k, directory_path = sentinel_path)
    break

Downloading: 100%|██████████| 775M/775M [16:57<00:00, 545kB/s]
MD5 checksumming: 100%|██████████| 775M/775M [00:01<00:00, 554MB/s]


AttributeError: 'collections.OrderedDict' object has no attribute 'index'

In [27]:
for file in os.listdir(sentinel_path):
    file_path = os.path.join(sentinel_path, file)
    out_path = os.path.join(sentinel_path, file.split('.')[0])
    if not os.path.exists(out_path):
        with zipfile.ZipFile(file_path,"r") as zip_ref:
            zip_ref.extractall(out_path)
            os.remove(file_path)
